### Gurobi Implementation

In [1]:
### imports
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from matplotlib import pyplot as plt

#### Set up model parameters

In [2]:
# regression results
regResults = pd.read_csv('regression_results.csv').drop(columns = ['Unnamed: 0'])
regResults.sort_values('p-value', ascending = True)

,feature,coefficient estimate,p-value
18,population_OtherRace,0.585061,0.000000e+00
14,population_White,0.268790,0.000000e+00
15,population_Black,0.233985,5.675460e-13
2,perc_prenat_1tri,0.073212,2.376876e-11
13,beds_per_1000,-0.110453,5.656041e-09
1,perc_csec,-0.110188,2.349227e-08
17,population_Asian,0.380202,1.047958e-05
10,hiv_tested,-0.043750,5.959245e-05
0,intercept,-28.916958,7.478606e-04
12,perc_obese,0.063540,2.134964e-03


In [3]:
# parameter Data
parametersData = pd.read_csv('parameters.csv').drop(columns = ['Unnamed: 0'])
parametersData.head()

,county_name,perc_csec,perc_prenat_1tri,birth_rate_15_19,birth_rate_20_24,birth_rate_25_29,birth_rate_30_34,birth_date_35_39,gono_per_100000,perc_smoker,hiv_tested,perc_no_healthins,perc_obese,beds_per_1000,population_White,population_Black,population_Native,population_Asian,population_OtherRace,ADI_STATERNK_INT_mean
0,Potter,35.9,77.8,27.7,145.7,135.1,81.7,29.4,75.71931,21.0,33.0,8.0,36.0,1.5,97.093197,0.341624,0.173809,0.419539,0.299670,8.222222
1,Wyoming,32.4,71.2,16.3,83.4,108.6,95.1,37.2,75.71931,24.0,45.0,11.0,29.0,0.4,93.101411,1.805894,0.169880,0.435778,0.324987,5.318182
2,Lehigh,29.9,73.9,19.8,70.9,104.0,105.2,49.3,102.80000,18.0,45.0,6.0,32.0,4.7,76.151936,7.268782,0.328852,3.331809,6.020613,4.940367
3,Indiana,24.0,65.7,9.5,41.8,128.1,98.0,38.7,34.70000,21.0,37.0,8.0,39.0,2.0,94.083800,2.260161,0.065117,0.928217,0.455821,7.158730
4,Schuylkill,32.1,67.6,20.9,88.7,114.6,90.6,36.6,34.50000,17.0,45.0,7.0,34.0,1.7,93.046113,2.989397,0.164864,0.421320,1.268186,7.559322


In [4]:
# decide program vars and filter parameter data and regression results
program_vars = ['birth_rate_15_19', 'perc_prenat_1tri', 'perc_csec', 'perc_smoker', 'perc_no_healthins']

# regression results for programs 
programWeights = regResults[regResults.feature.apply(lambda x: x in program_vars)].reset_index()
weightDict = {}
for var in program_vars:
    varWeight = programWeights.loc[np.where(programWeights.feature == var)[0][0], 'coefficient estimate']
    weightDict[var] = varWeight
    
print(weightDict)
    

# parameter rates for program vars 
programData = parametersData.copy()
programData = programData[program_vars]
#programData['gono_per_100000'] = programData['gono_per_100000']/100000
programData = programData.drop_duplicates().reset_index(drop = True)
print(programData.shape)
programData.head()

{'birth_rate_15_19': 0.0217538326349848, 'perc_prenat_1tri': 0.0732123029191958, 'perc_csec': -0.1101875123869329, 'perc_smoker': -0.0189638084573768, 'perc_no_healthins': -0.04380370470907}
(67, 5)


,birth_rate_15_19,perc_prenat_1tri,perc_csec,perc_smoker,perc_no_healthins
0,27.7,77.8,35.9,21.0,8.0
1,16.3,71.2,32.4,24.0,11.0
2,19.8,73.9,29.9,18.0,6.0
3,9.5,65.7,24.0,21.0,8.0
4,20.9,67.6,32.1,17.0,7.0


In [5]:
# get the non-program dot product

# nonprogram vars 
nonprogram_vars = [v for v in parametersData.columns.tolist() if v not in program_vars]
nonprogram_vars.remove('county_name')

# regression results for nonprogram vars 
nonprogramWeights = regResults[regResults.feature.apply(lambda x: x in nonprogram_vars)]
intercept = regResults[regResults.feature == 'intercept']['coefficient estimate'][0]

# parameter rates for non program vars 
nonprogramData = parametersData.copy()
nonprogramData = nonprogramData[nonprogram_vars]
nonprogramData.drop_duplicates(inplace = True)
nonprogramData.head()

nonprogramSum = []
for i in range(nonprogramData.shape[0]):
    countyNonProgramSum = nonprogramWeights.values[:,1].dot(nonprogramData.values[i,:])
    nonprogramSum.append(countyNonProgramSum+intercept)
    

len(nonprogramSum)



67

In [6]:
# define decision variables 
num_programs = len(program_vars)
num_counties = programData.shape[0]
counties = range(num_counties)
programs = range(num_programs)

# parameters--rates of outcomes of interest per county
# perhaps formatted as a matrix of row x col = number of counties x number of programs
#rates = np.array([67,num_programs])
rates = programData.values

# total goals
total_goals = 3

# equity constraint per county
# equity_cap = XXX

# parameter weights (from regression output)
program_weights = weightDict
weight_direction = [-1,1,-1,-1,-1] # (-1 if we want less of an outcome, 1 if we want more of an outcome, for each program)



#### Model setup

In [7]:
sum(nonprogramSum)

-48.849611508102875

In [8]:
j = 2
print(program_vars[j])
print(weight_direction[j])
print(program_weights[program_vars[j]])
i = 2
print(rates[i,j])
X = 1


program_weights[program_vars[j]]*((1+(weight_direction[j]*X))*rates[i,j])

perc_csec
-1
-0.1101875123869329
29.9


-0.0

In [9]:
# initialize model
model = gp.Model()

# decision variables
X = model.addVars(counties, programs)

# objective function
model.setObjective(sum(sum(program_weights[program_vars[j]]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) + nonprogramSum[i] for i in counties))

model.modelSense = GRB.MAXIMIZE

# constraints
for i in counties:
    #model.addConstr(sum(program_weights[j]*rates[i,j] for j in programs) - 
    #   sum(program_weights[j]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) >= equity_cap[i]) # equity constraint
    
    model.addConstr(sum(X[i,j] for j in programs) == 1) # program resource allocation constraint
    
    model.addConstr(sum(program_weights[program_vars[j]]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) + nonprogramSum[i] <= total_goals)
    model.addConstr(sum(program_weights[program_vars[j]]*((1+(weight_direction[j]*X[i,j]))*rates[i,j]) for j in programs) + nonprogramSum[i] >= 0)
    
    for j in programs: 
        model.addConstr(X[i,j] >= 0) # non-negativity
        model.addConstr(((1+(weight_direction[j]*X[i,j]))*rates[i,j]) >= 0) # ensure rates are between [0,1]
        model.addConstr(((1+(weight_direction[j]*X[i,j]))*rates[i,j]) <= 100) # ensure rates are between [0,100] since already as a percent

# optimize
model.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-10
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1206 rows, 335 columns and 2010 nonzeros
Model fingerprint: 0xeacec2a0
Coefficient statistics:
  Matrix range     [6e-02, 9e+01]
  Objective range  [6e-02, 6e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-03, 1e+02]
Presolve removed 1072 rows and 0 columns
Presolve time: 0.03s
Presolved: 134 rows, 353 columns, 688 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2784921e+02   7.994478e+01   0.000000e+00      0s
     198    2.0100000e+02   0.000000e+00   0.000000e+00      0s

Solved in 198 iterations and 0.05 seconds (0.00 work units)
Optimal objective  2.010000000e+02


In [10]:
model.ObjVal

200.99999999999991

In [128]:
program_vars

['birth_rate_15_19',
 'perc_prenat_1tri',
 'perc_csec',
 'perc_smoker',
 'perc_no_healthins']

In [11]:
# print allocations for first 10 counties as a check
for i in range(10):
    print('\nCounty ', i, ' allocations:')
    for j in programs: 
        print('Program ', j, ': ', X[i,j].x)


County  0  allocations:
Program  0 :  0.25033362729534664
Program  1 :  0.0
Program  2 :  0.7496663727046533
Program  3 :  0.0
Program  4 :  0.0

County  1  allocations:
Program  0 :  0.048178285967296564
Program  1 :  0.40449438202247184
Program  2 :  0.5473273320102316
Program  3 :  0.0
Program  4 :  0.0

County  2  allocations:
Program  0 :  0.3828208921478765
Program  1 :  0.0
Program  2 :  0.6171791078521235
Program  3 :  0.0
Program  4 :  0.0

County  3  allocations:
Program  0 :  0.26698711606855596
Program  1 :  0.0
Program  2 :  0.733012883931444
Program  3 :  0.0
Program  4 :  0.0

County  4  allocations:
Program  0 :  0.14413817924075822
Program  1 :  0.0
Program  2 :  0.8558618207592418
Program  3 :  0.0
Program  4 :  0.0

County  5  allocations:
Program  0 :  0.2275209543234522
Program  1 :  0.0
Program  2 :  0.7724790456765478
Program  3 :  0.0
Program  4 :  0.0

County  6  allocations:
Program  0 :  0.0
Program  1 :  0.0
Program  2 :  0.20744551543404147
Program  3 :  0

In [12]:
count = 0
for j in programs: 
    print(X[8,j].x)
    count +=X[8,j].x
    


0.02003560053674802
0.0
0.979964399463252
0.0
0.0


In [13]:
count


1.0